# Comparing Treatment Plans Across Time Points

This notebook demonstrates how to:
1. Load radiotherapy data from two different time points
2. Compare dose distributions between plans
3. Compare structure contours and detect geometric changes
4. Compute dosimetric differences (DVH, mean dose, max dose)
5. Visualize dose differences spatially

This is particularly useful for:
- Comparing initial vs adapted treatment plans
- Evaluating predicted vs delivered doses
- Longitudinal treatment analysis

## 1. Setup and Data Loading

First, download example data from HuggingFace which includes multiple time points.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from huggingface_hub import snapshot_download
from dosemetrics.io import load_structure_set, load_volume
from dosemetrics.dose import Dose
from dosemetrics.metrics import statistics, dvh, geometric

# Download test data (cached locally after first download)
data_path = Path(snapshot_download(
    repo_id="contouraid/dosemetrics-data",
    repo_type="dataset"
))
print(f"✓ Data downloaded to: {data_path}")


## 2. Load Data from Two Time Points

Load treatment plan data from two different time points (e.g., initial planning vs adaptive replanning).

In [ ]:
# For this example, we'll use test_subject data as "Time Point 1"
# and create a synthetic "Time Point 2" by loading the same data
# In practice, you would have two separate folders

time_point_1_path = data_path / "test_subject"
time_point_2_path = data_path / "test_subject"  # In practice, this would be different

# Load dose distributions
dose1, spacing1, origin1 = load_volume(time_point_1_path / "Dose.nii.gz")
dose2, spacing2, origin2 = load_volume(time_point_2_path / "Dose.nii.gz")

print("Time Point 1:")
print(f"  Dose shape: {dose1.shape}")
print(f"  Dose range: [{dose1.min():.2f}, {dose1.max():.2f}] Gy")
print(f"  Mean dose: {dose1.mean():.2f} Gy")

print("\nTime Point 2:")
print(f"  Dose shape: {dose2.shape}")
print(f"  Dose range: [{dose2.min():.2f}, {dose2.max():.2f}] Gy")
print(f"  Mean dose: {dose2.mean():.2f} Gy")

# Load structure sets
structures1 = load_structure_set(time_point_1_path)
structures2 = load_structure_set(time_point_2_path)

print(f"\nStructures at Time Point 1: {len(structures1)}")
print(f"Structures at Time Point 2: {len(structures2)}")

## 3. Compare Global Dose Statistics

First, let's look at overall dose distribution differences.

In [ ]:
# Compute dose difference
dose_diff = dose2 - dose1

# Global statistics
print("Global Dose Comparison:")
print(f"  Absolute mean difference: {np.abs(dose_diff).mean():.3f} Gy")
print(f"  RMS difference: {np.sqrt(np.mean(dose_diff**2)):.3f} Gy")
print(f"  Max difference: {np.abs(dose_diff).max():.3f} Gy")
print(f"  Correlation: {np.corrcoef(dose1.flatten(), dose2.flatten())[0,1]:.4f}")

# Create histogram comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Dose histograms
axes[0].hist(dose1[dose1 > 0].flatten(), bins=50, alpha=0.6, label='Time Point 1', density=True)
axes[0].hist(dose2[dose2 > 0].flatten(), bins=50, alpha=0.6, label='Time Point 2', density=True)
axes[0].set_xlabel('Dose (Gy)', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].set_title('Dose Distribution Comparison', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Difference histogram
axes[1].hist(dose_diff.flatten(), bins=100, alpha=0.7, color='purple')
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='No change')
axes[1].set_xlabel('Dose Difference (Gy)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Dose Difference Distribution', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Compare Structure-Specific Doses

Analyze dose differences within specific structures (PTV and OARs).

In [ ]:
# Compare doses in key structures
comparison_structures = ['PTV', 'Brainstem', 'Chiasm', 'OpticNerve_L', 'OpticNerve_R']

dose_comparison_data = []

for struct_name in comparison_structures:
    if struct_name not in structures1 or struct_name not in structures2:
        continue
    
    # Get masks
    mask1 = structures1.get_structure(struct_name).mask
    mask2 = structures2.get_structure(struct_name).mask
    
    # Compute metrics for time point 1
    mean1 = mean_dose(dose1, mask1)
    max1 = max_dose(dose1, mask1)
    
    # Compute metrics for time point 2
    mean2 = mean_dose(dose2, mask2)
    max2 = max_dose(dose2, mask2)
    
    # Compute differences
    mean_diff = mean2 - mean1
    max_diff = max2 - max1
    
    dose_comparison_data.append({
        'Structure': struct_name,
        'Mean_T1 (Gy)': mean1,
        'Mean_T2 (Gy)': mean2,
        'Mean_Diff (Gy)': mean_diff,
        'Max_T1 (Gy)': max1,
        'Max_T2 (Gy)': max2,
        'Max_Diff (Gy)': max_diff,
    })

# Create comparison DataFrame
comparison_df = pd.DataFrame(dose_comparison_data)
print("\\nDose Comparison by Structure:")
print(comparison_df.to_string(index=False))

# Visualize the comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean dose comparison
x_pos = np.arange(len(comparison_df))
width = 0.35

axes[0].bar(x_pos - width/2, comparison_df['Mean_T1 (Gy)'], width, 
           label='Time Point 1', alpha=0.8)
axes[0].bar(x_pos + width/2, comparison_df['Mean_T2 (Gy)'], width, 
           label='Time Point 2', alpha=0.8)
axes[0].set_xlabel('Structure', fontsize=12)
axes[0].set_ylabel('Mean Dose (Gy)', fontsize=12)
axes[0].set_title('Mean Dose Comparison', fontsize=14)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(comparison_df['Structure'], rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Max dose comparison
axes[1].bar(x_pos - width/2, comparison_df['Max_T1 (Gy)'], width, 
           label='Time Point 1', alpha=0.8)
axes[1].bar(x_pos + width/2, comparison_df['Max_T2 (Gy)'], width, 
           label='Time Point 2', alpha=0.8)
axes[1].set_xlabel('Structure', fontsize=12)
axes[1].set_ylabel('Max Dose (Gy)', fontsize=12)
axes[1].set_title('Max Dose Comparison', fontsize=14)
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(comparison_df['Structure'], rotation=45, ha='right')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Compare DVH Curves

Plot Dose-Volume Histograms (DVHs) for both time points to visualize changes.

In [ ]:
# Compare DVH for selected structures
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

structures_to_plot = ['PTV', 'Brainstem', 'Chiasm', 'OpticNerve_L']

for idx, struct_name in enumerate(structures_to_plot):
    if struct_name not in structures1:
        continue
    
    ax = axes[idx]
    
    # Get masks
    mask1 = structures1.get_structure(struct_name).mask
    mask2 = structures2.get_structure(struct_name).mask
    
    # Compute DVH
    bins1, values1 = compute_dvh(dose1, mask1, max_dose=70, step_size=0.1)
    bins2, values2 = compute_dvh(dose2, mask2, max_dose=70, step_size=0.1)
    
    # Plot
    ax.plot(bins1, values1, label='Time Point 1', linewidth=2, color='blue')
    ax.plot(bins2, values2, label='Time Point 2', linewidth=2, color='red', linestyle='--')
    ax.set_xlabel('Dose (Gy)', fontsize=11)
    ax.set_ylabel('Volume (%)', fontsize=11)
    ax.set_title(f'{struct_name} DVH Comparison', fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 70])
    ax.set_ylim([0, 105])

plt.tight_layout()
plt.show()

## 6. Compare Structure Geometries

Analyze geometric changes in structure contours between time points.

In [ ]:
# Compare structure volumes and overlaps
geometric_comparison = []

def compute_dice_coefficient(mask1, mask2):
    """Compute Dice similarity coefficient between two binary masks."""
    intersection = np.logical_and(mask1, mask2).sum()
    return 2.0 * intersection / (mask1.sum() + mask2.sum()) if (mask1.sum() + mask2.sum()) > 0 else 0.0

def compute_hausdorff_distance(mask1, mask2):
    """Approximate Hausdorff distance using centroid distance."""
    # Simplified version - compute centroid distance
    if mask1.sum() == 0 or mask2.sum() == 0:
        return np.nan
    
    centroid1 = np.array([np.mean(np.where(mask1)[i]) for i in range(3)])
    centroid2 = np.array([np.mean(np.where(mask2)[i]) for i in range(3)])
    return np.linalg.norm(centroid1 - centroid2)

for struct_name in comparison_structures:
    if struct_name not in structures1 or struct_name not in structures2:
        continue
    
    mask1 = structures1.get_structure(struct_name).mask
    mask2 = structures2.get_structure(struct_name).mask
    
    # Volume in voxels
    volume1 = mask1.sum()
    volume2 = mask2.sum()
    volume_change = volume2 - volume1
    volume_change_pct = (volume_change / volume1 * 100) if volume1 > 0 else 0
    
    # Dice coefficient
    dice = compute_dice_coefficient(mask1, mask2)
    
    # Centroid distance
    centroid_dist = compute_hausdorff_distance(mask1, mask2)
    
    geometric_comparison.append({
        'Structure': struct_name,
        'Volume_T1': volume1,
        'Volume_T2': volume2,
        'Volume_Change': volume_change,
        'Volume_Change_%': volume_change_pct,
        'Dice_Coefficient': dice,
        'Centroid_Distance': centroid_dist,
    })

geo_df = pd.DataFrame(geometric_comparison)
print("\\nGeometric Comparison:")
print(geo_df.to_string(index=False))

# Visualize geometric metrics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Volume change
axes[0].bar(geo_df['Structure'], geo_df['Volume_Change_%'], color='skyblue', alpha=0.8)
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0].set_xlabel('Structure', fontsize=12)
axes[0].set_ylabel('Volume Change (%)', fontsize=12)
axes[0].set_title('Structure Volume Changes', fontsize=14)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3, axis='y')

# Dice coefficient
axes[1].bar(geo_df['Structure'], geo_df['Dice_Coefficient'], color='lightgreen', alpha=0.8)
axes[1].axhline(y=0.8, color='orange', linestyle='--', linewidth=1, label='Good overlap (0.8)')
axes[1].set_xlabel('Structure', fontsize=12)
axes[1].set_ylabel('Dice Coefficient', fontsize=12)
axes[1].set_title('Structure Overlap (Dice)', fontsize=14)
axes[1].set_ylim([0, 1.05])
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Visualize Spatial Dose Differences

Show dose differences slice by slice to identify regions of change.

In [ ]:
# Visualize dose differences on selected slices
n_slices = 6
slice_indices = np.linspace(20, dose1.shape[0] - 20, n_slices, dtype=int)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, slice_idx in enumerate(slice_indices):
    ax = axes[idx]
    
    # Get the slice
    diff_slice = dose_diff[slice_idx, :, :]
    
    # Plot
    im = ax.imshow(diff_slice, cmap='RdBu_r', vmin=-5, vmax=5)
    ax.set_title(f'Slice {slice_idx}', fontsize=12)
    ax.axis('off')
    
    # Add colorbar
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

fig.suptitle('Dose Difference (T2 - T1) in Gy', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("\\nDose difference legend:")
print("  Red: Higher dose at Time Point 2")
print("  Blue: Lower dose at Time Point 2")
print("  White: No significant change")

## 8. Export Comparison Results

Save the comparison results to CSV files for further analysis.

In [ ]:
# Save comparison results
output_dir = Path("./comparison_results")
output_dir.mkdir(exist_ok=True)

# Save dose comparison
comparison_df.to_csv(output_dir / "dose_comparison.csv", index=False)
print(f"✓ Saved dose comparison to: {output_dir / 'dose_comparison.csv'}")

# Save geometric comparison
geo_df.to_csv(output_dir / "geometric_comparison.csv", index=False)
print(f"✓ Saved geometric comparison to: {output_dir / 'geometric_comparison.csv'}")

print("\\n✓ All comparison results exported successfully!")

## Summary

In this notebook, you learned how to:

1. ✓ Load radiotherapy data from multiple time points
2. ✓ Compare global dose distribution statistics
3. ✓ Analyze structure-specific dose changes
4. ✓ Compare DVH curves across time points
5. ✓ Evaluate geometric changes in structure contours (volume, Dice coefficient)
6. ✓ Visualize spatial dose differences
7. ✓ Export comparison results for reporting

## Key Metrics for Plan Comparison

**Dosimetric Metrics:**
- Mean dose per structure
- Maximum dose per structure
- DVH curves and metrics
- Global dose statistics (RMS, correlation)

**Geometric Metrics:**
- Volume changes
- Dice similarity coefficient (overlap)
- Centroid distance
- Hausdorff distance

## Real-World Applications

This workflow is valuable for:
- **Adaptive Radiotherapy**: Comparing initial vs adapted plans
- **Quality Assurance**: Verifying predicted vs delivered doses
- **Clinical Research**: Analyzing treatment response and anatomical changes
- **Machine Learning**: Validating dose prediction models

## Next Steps

- **Quality Indices**: Compute conformity indices and compliance metrics
- **Advanced Metrics**: Gamma analysis, dose gradients
- **Batch Processing**: Automate comparison for multiple patients
- **Statistical Analysis**: Perform population-level comparisons

## References

- [DoseMetrics Documentation](https://contouraid.github.io/dosemetrics/)
- [GitHub Repository](https://github.com/contouraid/dosemetrics)